# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [26]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [27]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [28]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [29]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [30]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [31]:
# Check the result:

A @ x - b

array([ 0.00000000e+00,  2.22044605e-16,  0.00000000e+00, -1.11022302e-16,
        0.00000000e+00,  0.00000000e+00, -2.08166817e-17,  0.00000000e+00,
        0.00000000e+00,  2.22044605e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [57]:
#копируем уже готовые куски кода, потому что они работают
def Jacobi(A, b, n_iter): 
    diag_1d = np.diag(A)

    B = -A.copy()
    np.fill_diagonal(B, 0)
    
    D = np.diag(diag_1d)
    invD = np.diag(1./diag_1d)
    BB = invD @ B 
    c = invD @ b
    
    x0 = np.ones(n)
    x = x0
    for _ in range(n_iter):
        x = BB @ x + c
    return x, np.linalg.norm(BB)

In [58]:
n_iter = 1000
x1, norm_BB = Jacobi(A, b, n_iter)
A @ x1 - b

array([ 0.00000000e+00,  2.22044605e-16,  0.00000000e+00, -1.11022302e-16,
        0.00000000e+00,  0.00000000e+00, -2.08166817e-17,  0.00000000e+00,
        0.00000000e+00,  2.22044605e-16])

In [68]:
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([20]*n)
b = rndm.uniform(size=n)
n_iter = 1000
x1, norm_BB = Jacobi(A, b, n_iter)
print (norm_BB)
print (A@x1 - b)

0.2752484728535418
[ 0.00000000e+00  1.11022302e-16  1.11022302e-16  0.00000000e+00
 -1.11022302e-16  0.00000000e+00  0.00000000e+00  1.11022302e-16
  0.00000000e+00  0.00000000e+00]


In [59]:
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)
n_iter = 1000
x1, norm_BB = Jacobi(A, b, n_iter)
print (norm_BB)
print (A@x1 - b)

0.36436161983015336
[ 0.00000000e+00  2.22044605e-16  0.00000000e+00 -1.11022302e-16
  0.00000000e+00  0.00000000e+00 -2.08166817e-17  0.00000000e+00
  0.00000000e+00  2.22044605e-16]


In [60]:
rndm = np.random.RandomState(2345)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([5]*n)
b = rndm.uniform(size=n)
n_iter = 1000
x1, norm_BB = Jacobi(A, b, n_iter)
print (norm_BB)
print (A@x1 - b)

1.0045798815149969
[-3.46944695e-18  1.11022302e-16  5.55111512e-17  0.00000000e+00
  1.11022302e-16  3.46944695e-17  1.11022302e-16  1.38777878e-17
  0.00000000e+00  1.38777878e-17]


In [62]:
rndm = np.random.RandomState(3456)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([3]*n)
b = rndm.uniform(size=n)
n_iter = 1000
x1, norm_BB = Jacobi(A, b, n_iter)
print (norm_BB)
print (A@x1 - b)

1.682478753503419
[1.70982690e+131 1.71907367e+131 2.62562773e+131 1.67733293e+131
 1.94802772e+131 1.80501362e+131 1.67188076e+131 1.90858226e+131
 1.93584120e+131 1.72837434e+131]


Заметим, что когда диагональные элементы матрицы перестают доминировать, метод ломается 

# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [241]:
# ... ENTER YOUR CODE HERE ...

def Seidel(A, b, n_iter):
    n = len(b)
    x = np.ones(n)

    for _ in range(n_iter):
        for i in range(n):
            x[i] = (-sum(A[i, j]*x[j] for j in range(n)) + b[i])/A[i, i] + x[i]
    return x

In [242]:
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)
n_iter = 1000
x = Seidel(A, b, n_iter)
print(np.linalg.norm(A@x - b))

2.7485378307127106e-16


In [243]:
rndm = np.random.RandomState(1437)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([6]*n)
b = rndm.uniform(size=n)
n_iter = 1000
x = Seidel(A, b, n_iter)
print(np.linalg.norm(A@x - b))

2.884444029575346e-16


In [244]:
rndm = np.random.RandomState(2735)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([27]*n)
b = rndm.uniform(size=n)
n_iter = 1000
x = Seidel(A, b, n_iter)
print(np.linalg.norm(A@x - b))

1.3597399555105182e-16


Вроде сходится

# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [ ]:
# ... ENTER YOUR CODE HERE ...